# Set up workspace

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd
from scipy.stats import ttest_ind
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
#from eofs.xarray import Eof 

In [2]:
import cftime
import scipy

In [3]:
import sys
sys.path.append('/glade/u/home/czarakas/coupled_PPE/code/utils')

import make_multisimulation_dataset

In [4]:
from load_ensembles import *
import quick_map

In [5]:
#path_out='/glade/work/czarakas/coupled_PPE/data/data_for_figures/'
path_out='~/coupled_PPE/data/processed_data_for_figures/'

# Load data

### Define variables to load

In [6]:
var='TSKIN'
domain='lnd'
ensemble='coupled'
season='Annual'

In [7]:
end_spinup=40

### Load full ensemble data

In [8]:
if ensemble=='coupled':
    ensemble_coupled = load_coupled_ensemble(var=var, domain=domain, printon=False)
elif ensemble=='offline':
    ensemble_coupled = load_offline_ensemble(var=var, domain=domain, printon=False)

/glade/u/home/czarakas/coupled_PPE/code/utils/load_ensembles.py:46: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  ds = xr.open_mfdataset(fpath)
/glade/u/home/czarakas/.conda/envs/EOF_env/lib/python3.7/site-packages/xarray/backends/api.py:941: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


In [9]:
if ensemble=='offline':
    ensemble_path='offline_simulations'
    refcase_id='OFFL0000_PI_v02'
elif ensemble=='coupled':
    ensemble_path='coupled_simulations'
    refcase_id='COUP0000_PI_SOM'
if domain=='atm':
    domain_suffix='.cam.h0.'
elif domain=='lnd':
    domain_suffix='.clm2.h0.'
    
basecase_lnd=xr.open_dataset('/glade/campaign/cgd/tss/czarakas/CoupledPPE/'+ensemble_path+'/'+
                                 refcase_id+'/'+domain+'/proc/tseries/'+refcase_id+domain_suffix+'timeseries.'+var+'.nc')

### Define function for calculating annual averages based on days per year

In [10]:
dpm = {'noleap': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '365_day': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'standard': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'proleptic_gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'all_leap': [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '366_day': [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '360_day': [0, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]}

def get_dpm(time, calendar='standard'):
    """
    return a array of days per month corresponding to the months provided in `months`
    """
    month_length = np.zeros(len(time), dtype=np.int)

    cal_days = dpm[calendar]

    for i, (month, year) in enumerate(zip(time.month, time.year)):
        month_length[i] = cal_days[month]
    return month_length

In [11]:
ds=basecase_lnd
month_length = xr.DataArray(get_dpm(ds.time.to_index(),
                                      calendar='noleap'),
                              coords=[ds.time], name='month_length')

numyears=np.size(ds.time.groupby('time.year').mean(dim='time').year)

In [12]:
# Calculate the weights by grouping by 'time.season'.
# Conversion to float type ('astype(float)') only necessary for Python 2.x
weights = month_length.groupby('time.year') / month_length.astype(float).groupby('time.year').sum()

# Test that the sum of the weights for each season is 1.0
#weights.groupby('time.month').sum().values
np.testing.assert_allclose(weights.groupby('time.year').sum().values, np.ones(numyears))

# Calculate the weighted average
ds_weighted = (ds[var] * weights).groupby('time.year').sum(dim='time')

In [13]:
def calculate_annual_timeseries(da, season='Annual',
                               end_spinup=0):
    month_length = xr.DataArray(get_dpm(ds.time.to_index(),
                                      calendar='noleap'),
                              coords=[ds.time], name='month_length')
    
    numyears=np.size(ds.time.groupby('time.year').mean(dim='time'))
    
    if season=='Annual':
        weights = month_length.groupby('time.year') / month_length.astype(float).groupby('time.year').sum()
        
        # Test that the sum of the weights for each year is 1.0
        np.testing.assert_allclose(weights.groupby('time.year').sum().values, np.ones(numyears))
    
        # Calculate the weighted average
        da_weighted = (da * weights).groupby('time.year').sum(dim='time')
        
        this_ensemble_tseries=da_weighted[end_spinup:,:,:]
    else:
        months_filtered = month_length.where(month_length['time.season']==season)
        weights = months_filtered.groupby('time.year') / months_filtered.astype(float).groupby('time.year').sum()
        
        # Test that the sum of the weights for each season is 1.0
        np.testing.assert_allclose(weights.groupby('time.year').sum().values, np.ones(numyears))
        
        da_weighted = (da * weights).groupby('time.year').sum(dim='time')
        this_ensemble_tseries=da_weighted[end_spinup:,:,:]
        
    return this_ensemble_tseries
    

### Check ensemble

In [17]:
ensemble_coupled[-3]#.FSH

<xarray.Dataset>
Dimensions:      (hist_interval: 2, lat: 96, lon: 144, time: 1680)
Coordinates:
  * time         (time) object 0049-01-16 12:00:00 ... 0188-12-16 12:00:00
  * lon          (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat          (lat) float32 -90.0 -88.10526 -86.210526 ... 88.10526 90.0
Dimensions without coordinates: hist_interval
Data variables:
    time_bounds  (time, hist_interval) object dask.array<chunksize=(1680, 2), meta=np.ndarray>
    TSKIN        (time, lat, lon) float32 dask.array<chunksize=(1680, 96, 144), meta=np.ndarray>
    var          (time, lat, lon) float32 dask.array<chunksize=(1680, 96, 144), meta=np.ndarray>
Attributes:
    title:                                     CLM History file information
    comment:                                   NOTE: None of the variables ar...
    Conventions:                               CF-1.0
    history:                                   Fri Jul  8 20:44:13 2022: ncrc...
    source:                                    Community Terrestrial Systems ...
    hostname:                                  cheyenne
    username:                                  czarakas
    version:                                   cesm2.2.0
    revision_id:                               $Id: histFileMod.F90 42903 201...
    case_title:                                UNSET
    case_id:                                   COUP0006_PI_SOM_v02
    Surface_dataset:                           surfdata_1.9x2.5_hist_16pfts_I...
    Initial_conditions_dataset:                arbitrary initialization
    PFT_physiological_constants_dataset:       OAAT0034.nc
    ltype_vegetated_or_bare_soil:              1
    ltype_crop:                                2
    ltype_UNUSED:                              3
    ltype_landice_multiple_elevation_classes:  4
    ltype_deep_lake:                           5
    ltype_wetland:                             6
    ltype_urban_tbd:                           7
    ltype_urban_hd:                            8
    ltype_urban_md:                            9
    ctype_vegetated_or_bare_soil:              1
    ctype_crop:                                2
    ctype_crop_noncompete:                     2*100+m, m=cft_lb,cft_ub
    ctype_landice:                             3
    ctype_landice_multiple_elevation_classes:  4*100+m, m=1,glcnec
    ctype_deep_lake:                           5
    ctype_wetland:                             6
    ctype_urban_roof:                          71
    ctype_urban_sunwall:                       72
    ctype_urban_shadewall:                     73
    ctype_urban_impervious_road:               74
    ctype_urban_pervious_road:                 75
    cft_c3_crop:                               1
    cft_c3_irrigated:                          2
    time_period_freq:                          month_1
    Time_constant_3Dvars_filename:             ./COUP0006_PI_SOM_v02.clm2.h3....
    Time_constant_3Dvars:                      ZSOI:DZSOI:WATSAT:SUCSAT:BSW:H...
    NCO:                                       netCDF Operators version 5.0.3...

In [15]:
1716/12

143.0

In [18]:
ensemble_coupled[-1]=ensemble_coupled[-1].sel(time=slice('0049-01-16', '0188-12-16'))

In [19]:
expected_start_time=cftime.DatetimeNoLeap(49,1,16,12)
expected_end_time=cftime.DatetimeNoLeap(188,12,16,12)
for i,ds in enumerate(ensemble_coupled):
    #print(i)
    #print(ds.case_id)
    if np.size(ds.time)!=1680:
        print(ds.case_id)
        print('!!!!! Time size is wrong dimension')
    if ds.time.values[0] != expected_start_time:
        print(ds.time.values[0])
    if ds.time.values[-1] != expected_end_time:
        print(ds.time.values[-1])
    #print('----------')

# Make datasets with all data

In [20]:
# Define grid for dataset
ds_grid = ensemble_coupled[0]

# Make data arrays
var_array_1time = make_multisimulation_dataset.make_empty_dataarray(ds_grid=ds_grid, var=var, keys=keys)

var_array=var_array_1time.expand_dims({"time": ds_grid.time[end_spinup*12:]},axis=0).copy()

/glade/u/home/czarakas/.conda/envs/EOF_env/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [ ]:
# Put data in data arrays
for i, ds in enumerate(ensemble_coupled):
    print(i)
    var_array[:,:,:,i]=ds[var][end_spinup*12:,:,:]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [ ]:
var_ref_annual=calculate_annual_timeseries(basecase_lnd[var][end_spinup*12:,:,:],season=season)
var_ref_mean=var_ref_annual.mean(dim='year')
ds_ref=var_ref_mean.to_dataset(name=var)

In [ ]:
var_array_annual=calculate_annual_timeseries(var_array,season=season)
var_array_mean=var_array_annual.mean(dim='year')
ds_mean=var_array_mean.to_dataset(name=var)

In [ ]:
var_array_delta=var_array_mean-var_ref_mean
ds_delta=var_array_delta.to_dataset(name=var)

In [ ]:
fname=var+'_'+season+'_mean_'+ensemble+'.nc'
ds_mean.to_netcdf(path_out+fname)

fname='ref_'+var+'_'+season+'_mean_'+ensemble+'.nc'
ds_ref.to_netcdf(path_out+fname)

fname='delta_'+var+'_'+season+'_mean+'+ensemble+'.nc'
ds_delta.to_netcdf(path_out+fname)

# Calculate where changes are significant

In [23]:
ds_grid = ensemble_coupled[0]

var_array_ttest = make_multisimulation_dataset.make_empty_dataarray(ds_grid=ds_grid, var=var, keys=keys)
var_array_pval = make_multisimulation_dataset.make_empty_dataarray(ds_grid=ds_grid, var=var, keys=keys)

/glade/u/home/czarakas/.conda/envs/EOF_env/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/u/home/czarakas/.conda/envs/EOF_env/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [24]:
for i, ds in enumerate(ensemble_coupled):
    ds_annual=var_array_annual[:,:,:,i]
    [tstat, pval] = scipy.stats.ttest_ind(var_ref_annual, ds_annual, equal_var=False)#, alternative='two-sided')
    var_array_ttest[:,:,i]=tstat
    var_array_pval[:,:,i]=pval

/glade/u/home/czarakas/.conda/envs/EOF_env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/glade/u/home/czarakas/.conda/envs/EOF_env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/glade/u/home/czarakas/.conda/envs/EOF_env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [25]:
ds_pval=var_array_pval.to_dataset(name='pval')

fname='pval_'+var+'_'+season+'_mean_'+ensemble+'.nc'
ds_pval.to_netcdf(path_out+fname)